IMPORTS

In [ ]:
# Imports
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Optional, Tuple
from deap import base, creator, tools

# File name
FILENAME = "tsp.dat"

HELPER FUNCTIONS

In [ ]:
def haversine(lat_p1, long_p1, lat_p2, long_p2):
    Earth_radius = 3958.88      # miles
    # Convert to radian for math. functions
    phi_1 = math.radians(lat_p1)
    phi_2 = math.radians(lat_p2)
    lambda_p1 = math.radians(long_p1)
    lambda_p2 = math.radians(long_p2)

    dphi = phi_2 - phi_1
    dlambda = lambda_p2 - lambda_p1

    hav_dphi = math.pow(math.sin(dphi/2), 2)
    cos_phi1 = math.cos(phi_1)
    cos_phi2 = math.cos(phi_2)
    hav_dlambda = math.pow(math.sin(dlambda/2), 2)

    hav_theta = hav_dphi + cos_phi1*cos_phi2*hav_dlambda
    theta = 2 * math.asin(math.sqrt(hav_theta))

    return theta * Earth_radius

def haversine_matrix(coordinates):
    return

def evaluate_TSP(ind):
    return

def compare_with_best(best_min_distance):
    # Best minimum distance is 10,637.36 miles
    optimum = 10637.36
    # Compare our best with the true best and represent as a %
    per = ((best_min_distance - optimum) / optimum ) * 100
    return

def load_dat(file):
    cities = []
    # Latitude Longitude Coordinates
    coordinates = []

    with open(file, "r") as f:
        for capital_cities in f:
            # Extract information
            parts = capital_cities.strip().split()

            # FORMAT: Albany, NY          42.652552778 -73.75732222
            city_state = " ".join(parts[:-2])
            latitude = float(parts[-2])
            longitude = float(parts[-1])

            # Append to list
            cities.append(city_state)
            coordinates.append((latitude, longitude))

    return cities, coordinates

Genetic Algorithm (GA) Configuration and Implementation

In [ ]:
# DEAP setup
def ensure_deap_creators() -> None:
    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))     # We are MINIMIZING
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)
        
# ===================== GA Configuration Dataclass =====================
# Configuration dataclass
@dataclass(frozen=True)         # Parameters cant be changed during runs
class Config:
    genome_length: int = 49     # 49 indices of city, cooridnates
    pop_size: int = 500         # n individuals
    generations: int = 1000     # Allowed generations
    cxpb: float = 0.7           # crossover prob
    mutpb: float = 0.15         # mutation probability
    tournsize: int = 3          # For tournament slection
    eliteSize: int = 2          # Best k individuals to transfer to next gen
    seed: Optional[int] = None

# Build DEAP toolbox
def build_toolbox(cfg: Config) -> base.Toolbox:
    ensure_deap_creators()

    toolbox = base.Toolbox()
    toolbox.register("indices", random.sample, range(cfg.genome_length), cfg.genome_length)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutInversion)
    toolbox.register("select", tools.selTournament, tournsize=cfg.tournsize)

    return toolbox


# Single run of the GA
def run(cfg: Config) -> Dict[str, Any]:
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)

    # List of city names and coordinates
    cities, coordinates = load_dat(FILENAME)

    Haversine_Dist_matrix = haversine_matrix(coordinates)

    # Setup toolbox
    toolbox = build_toolbox(cfg)
    toolbox.register("evaluate", evaluate_TSP(Haversine_Dist_matrix))
    pop = toolbox.population(n=cfg.pop_size)    # Create initial popultation

    # Evaluate initial population (gen=0)
    fitnesses = map(toolbox.evaluate, pop)  # Compute fitness for each ind
    for ind, fit in zip(pop, fitnesses):    # Assign fitness back to ind
        ind.fitness.values = fit

    # Records Global Best solution, so that it is not lost
    hall_of_fame = tools.HallOfFame(cfg.eliteSize)
    
    # Link current statistical information we want for each indivudal
    statistics = tools.Statistics(lambda ind: ind.fitness.values[0])    # Get each ind's fitness value
    statistics.register("min", np.min)
    statistics.register("mean", np.mean)
    statistics.register("std", np.std)

    # Make a logbook for future reference
    logbook = tools.Logbook()
    logbook.header = ("gen", "nevals", "mean", "min", "std")

    # Fill in logbook w/ initial info
    stats = statistics.compile(pop)
    logbook.record(gen=0, nevals=len(pop), **stats)

    hall_of_fame.update(pop)
    # Evolution loop
    for gen in range(1, cfg.generations + 1):
        # Select offspring
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cfg.cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation
        for mutant in offspring:
            if random.random() < cfg.mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate individuals with invalid fitness
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid)      # Produce new fitness values
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        # Elitism update if any new global best made from offspring
        hall_of_fame.update(offspring)

        # Inject hall of fame ind back into population
        offspring.sort(key=lambda ind: ind.fitness.values[0])
        for ind in range(cfg.eliteSize):
            offspring[ind] = toolbox.clone(hall_of_fame[ind])     
        
        pop[:] = offspring

        # Update Logbook for current generation
        stats = statistics.compile(pop)
        logbook.record(gen=gen, nevals=len(pop), **stats)

        # For console printing
        print(logbook.stream)
    
    best_ind = hall_of_fame[0]
    best_distance = best_ind.fitness.values[0]

    return {
        "best_distance": best_distance,
        "best_tour": best_ind,
        "logbook": logbook
    }
        

MAIN

In [ ]:
def main():
    cfg = Config()
    results = run(cfg)

    logbook = results["logbook"]    # access the logbook
    best_distance = results["best_distance"]
    best_ind = results["best_tour"]
    print(logbook)

    print("\n===========================================")
    print("SHORTES PATH TOUR OBTAINED")
    print("===========================================")
    print(f"Best Minimum Distance Found = {best_distance} miles\n")
    print("Best Tour Found:", best_ind)



In [ ]:
if __name__ == "__main__":
    main()